In [ ]:
# Group Number : 6
# Roll Numbers : 19EC37002, 20CS10047
# Project Number : Khyathi Nalluri, Priyanshi Dixit
# Project Title : [ Project Code: TSDT ] - Predicting Sinking Titanic Survivors using Decision Tree based Learning Model

In [1]:
import pandas as pd
import numpy as np
from pprint import pprint
import csv # data processing, CSV file I/O
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier # sklearn Decision Tree
import operator
from math import log
from collections import Counter
from sklearn.metrics import confusion_matrix

class decision_tree():
    def __init__(self, train, test, prune=False):
        self.train = train
        self.test = test
        self.prune = prune
        self.column_headers = self.train.columns
        self.tree = self.decision_tree_algorithm_igain(self.train)
        self.accuracy = self.calculate_accuracy(self.tree)
        if prune :
            self.pruned_tree = self.post_pruning(self.tree, self.train, self.test)
            self.pruned_accuracy = self.calculate_accuracy(self.pruned_tree)


    def check_purity(self,data):
        label_column = data[:, -1]
        unique_classes = np.unique(label_column)

        if len(unique_classes) == 1:
            return True
        else:
            return False

    def classify_data(self,data):
        label_column = data[:, -1]
        unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

        index = counts_unique_classes.argmax()
        classification = unique_classes[index]

        return classification

    def get_potential_splits(self,data):
        potential_splits = {}
        _, n_columns = data.shape
        for column_index in range(n_columns - 1):  # excluding the last column which is the label
            potential_splits[column_index] = []
            values = data[:, column_index]
            unique_values = np.unique(values)

            for index in range(len(unique_values)):
                if index != 0:
                    current_value = unique_values[index]
                    previous_value = unique_values[index - 1]
                    potential_split = (current_value + previous_value) / 2

                    potential_splits[column_index].append(potential_split)

        return potential_splits

    def split_data(self,data, split_column, split_value):
        split_column_values = data[:, split_column]

        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values > split_value]

        return data_below, data_above

    def calculate_entropy(self,data):
        label_column = data[:, -1]
        _, counts = np.unique(label_column, return_counts=True)

        probabilities = counts / counts.sum()
        entropy = sum(probabilities * -np.log2(probabilities))

        return entropy

    def calculate_overall_entropy(self,data_below, data_above):
        n = len(data_below) + len(data_above)
        p_data_below = len(data_below) / n
        p_data_above = len(data_above) / n

        overall_entropy = (p_data_below * self.calculate_entropy(data_below)
                           + p_data_above * self.calculate_entropy(data_above))

        return overall_entropy

    def determine_best_split_igain(self,data, potential_splits):
        overall_entropy = 9999
        for column_index in potential_splits:
            for value in potential_splits[column_index]:
                data_below, data_above = self.split_data(data, split_column=column_index, split_value=value)
                current_overall_entropy =self.calculate_overall_entropy(data_below, data_above)

                if current_overall_entropy <= overall_entropy:
                    overall_entropy = current_overall_entropy
                    best_split_column = column_index
                    best_split_value = value

        return best_split_column, best_split_value

    def decision_tree_algorithm_igain(self, df, counter=0, min_samples=2, max_depth=6):
        # data preparations
        if counter == 0:
            data = df.values
        else:
            data = df

            # base cases
        if (self.check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
            classification = self.classify_data(data)

            return classification

        # recursive part
        else:
            counter += 1
            # helper functions
            potential_splits = self.get_potential_splits(data)
            split_column, split_value = self.determine_best_split_igain(data, potential_splits)
            data_below, data_above = self.split_data(data, split_column, split_value)
            # instantiate sub-tree
            feature_name = self.column_headers[split_column]
            question = "{} <= {}".format(feature_name, split_value)
            sub_tree = {question: []}

            # find answers (recursion)
            yes_answer = self.decision_tree_algorithm_igain(data_below, counter, min_samples, max_depth)
            no_answer = self.decision_tree_algorithm_igain(data_above, counter, min_samples, max_depth)

            # If the answers are the same, then there is no point in asking the qestion.
            # This could happen when the data is classified even though it is not pure
            # yet (min_samples or max_depth base cases).
            if yes_answer == no_answer:
                sub_tree = yes_answer
            else:
                sub_tree[question].append(yes_answer)
                sub_tree[question].append(no_answer)

            return sub_tree

    def calculate_accuracy(self, tree):

        def classify_example(example, tree):
            question = list(tree.keys())[0]
            feature_name, comparison_operator, value = question.split(" ")

            # ask question
            if comparison_operator == "<=":  # feature is continuous
                if example[feature_name] <= float(value):
                    answer = tree[question][0]
                else:
                    answer = tree[question][1]

            # feature is categorical
            else:
                if str(example[feature_name]) == value:
                    answer = tree[question][0]
                else:
                    answer = tree[question][1]

            # base case
            if not isinstance(answer, dict):
                return answer

            # recursive part
            else:
                residual_tree = answer
                return classify_example(example, residual_tree)

        adf = self.test.copy(deep=True)
        adf.insert(1, "classification", adf.apply(classify_example, axis=1, args=(tree, )).to_list())
        adf.insert(1, "classification_correct", list(adf["classification"] == adf["label"]))
        accuracy = adf["classification_correct"].mean()
        
        print("Confusion Matrix: ")
        print(confusion_matrix(adf['label'], adf['classification']))
    
        print("Accuracy: ")
        print(accuracy*100)
    
        print("Report: ")
        print(classification_report(adf['label'], adf['classification']))
        
        return accuracy

    # pruning methods

    def make_predictions(self, df, tree):

        def predict_example(example, tree):
            # tree is just a root node
            if not isinstance(tree, dict):
                return tree

            question = list(tree.keys())[0]
            feature_name, comparison_operator, value = question.split(" ")

            # ask question
            if comparison_operator == "<=":
                if example[feature_name] <= float(value):
                    answer = tree[question][0]
                else:
                    answer = tree[question][1]

            # feature is categorical
            else:
                if str(example[feature_name]) == value:
                    answer = tree[question][0]
                else:
                    answer = tree[question][1]

            # base case
            if not isinstance(answer, dict):
                return answer

            # recursive part
            else:
                residual_tree = answer
                return predict_example(example, residual_tree)

        if len(df) != 0:
            predictions = df.apply(predict_example, args=(tree,), axis=1)
        else:
            # "df.apply()"" with empty dataframe returns an empty dataframe,
            # but "predictions" should be a series instead
            predictions = pd.Series(dtype='float64')
            

        return predictions

    def filter_df(self, df, question):
        feature, comparison_operator, value = question.split()

        # continuous feature
        if comparison_operator == "<=":
            df_yes = df[df[feature] <= float(value)]
            df_no = df[df[feature] > float(value)]

        # categorical feature
        else:
            df_yes = df[df[feature].astype(str) == value]
            df_no = df[df[feature].astype(str) != value]

        return df_yes, df_no

    def determine_leaf(self, df_train):
        return df_train.label.value_counts().index[0]

    def determine_errors(self, df_val, tree):
        predictions = self.make_predictions(df_val, tree)
        actual_values = df_val.label
        return sum(predictions != actual_values)

    def pruning_result(self, tree, df_train, df_val):
        leaf = self.determine_leaf(df_train)
        errors_leaf = self.determine_errors(df_val, leaf)
        errors_decision_node = self.determine_errors(df_val, tree)

        if errors_leaf <= errors_decision_node:
            return leaf
        else:
            return tree

    def post_pruning(self, tree, df_train, df_val):
        question = list(tree.keys())[0]
        yes_answer, no_answer = tree[question]

        # base case
        if not isinstance(yes_answer, dict) and not isinstance(no_answer, dict):
            return self.pruning_result(tree, df_train, df_val)

        # recursive part
        else:
            df_train_yes, df_train_no = self.filter_df(df_train, question)
            df_val_yes, df_val_no = self.filter_df(df_val, question)

            if isinstance(yes_answer, dict):
                yes_answer = self.post_pruning(yes_answer, df_train_yes, df_val_yes)

            if isinstance(no_answer, dict):
                no_answer = self.post_pruning(no_answer, df_train_no, df_val_no)

            tree = {question: [yes_answer, no_answer]}

            return self.pruning_result(tree, df_train, df_val)


def get_df(file):
    # loading data into a dataframe
    df = pd.read_csv(file)

    # dropping unwanted columns from DF
    cols_to_drop = ['PassengerId', 'Name', 'Ticket', 'Fare', 'Cabin', 'Embarked']
    df = df.drop(cols_to_drop, axis=1)

    from sklearn.preprocessing import LabelEncoder
    encoder = LabelEncoder()
    df['Sex'] = encoder.fit_transform(df['Sex'])

    # Since we have null values only for the Age column,we will replace
    # the null values with the mean
    df = df.fillna(df['Age'].mean())
    df = df.rename(columns={"Survived": "label"})
    df = df[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'label']]
    return df

def get_train_test_split_kfold(data_df, no_of_splits=5):
    from random import shuffle
    # get indices from data
    indices = data_df.index.to_list()
    shuffle(indices)
    grouped_indices = [indices[i::no_of_splits] for i in range(no_of_splits)]
    test_train_batches = []#{'batch-{}'.format(i + 1): None for i in range(no_of_splits)}
    for j in range(no_of_splits):
        test_df = data_df.iloc[grouped_indices[j]]
        train_df = data_df.drop(grouped_indices[j])
        # test_train_batches['batch-{}'.format(j + 1)] = [train_df, test_df]
        test_train_batches.append((train_df, test_df))
    return test_train_batches


if __name__ == '__main__' :
    file = r".\titanic.csv"
    data_df = get_df(file)
    column_headers = data_df.columns
    test_train_batches = get_train_test_split_kfold(data_df, no_of_splits=5)
    trees = []
    pruned_trees = []
    accuracies = []
    pruned_accuracies = []
    best_accuracy, idx1 = 0, 0
    from tqdm import tqdm
    for i, data in tqdm(enumerate(test_train_batches)):
        train, test = data
        y_test = test.iloc[:,0]
        tree = decision_tree(train, test, prune=False)
        trees.append(tree)
        # 1
        # pruned_trees.append(tree.pruned_tree)
        if tree.accuracy > best_accuracy:
            best_accuracy = tree.accuracy
            idx = i
        accuracies.append(tree.accuracy)
        # 1
        # pruned_accuracies.append(tree.pruned_accuracy)

    best_tree = trees[idx]
    pruned_best_tree = best_tree.post_pruning(best_tree.tree, best_tree.train, best_tree.test)
    pruned_accuracy = best_tree.calculate_accuracy(pruned_best_tree)

    print('accuracies of Trees corresponding to 5 data splits :')
    for i, accuracy in enumerate(accuracies):
        print('tree-{} : {}'.format(i+1, round(accuracy, 3)))

    # 1
    # print('accuracies of Trees corresponding to 5 data splits after pruning:')
    # for i, accuracy in enumerate(pruned_accuracies):
    #     print('tree-{} : {}'.format(i + 1, round(accuracy, 3)))

    # 2
    print('tree with best accuracy - {} :'.format(round(best_accuracy, 4)))
    pprint(trees[idx].tree)

    # 1
    # print('best tree with best accuracy - {} after pruning [pruned accuracy - {}] :'.format(round(best_accuracy, 2), round(pruned_accuracies[idx], 2)))
    # pprint(pruned_trees[idx])

    # 2
    print('best tree with best accuracy - {} after pruning [pruned accuracy - {}] :'.format(round(best_accuracy, 2), round(pruned_accuracy, 2)))
    pprint(pruned_best_tree)
    

2it [00:00,  7.31it/s]

Confusion Matrix: 
[[101  15]
 [ 15  48]]
Accuracy: 
83.24022346368714
Report: 
              precision    recall  f1-score   support

           0       0.87      0.87      0.87       116
           1       0.76      0.76      0.76        63

    accuracy                           0.83       179
   macro avg       0.82      0.82      0.82       179
weighted avg       0.83      0.83      0.83       179

Confusion Matrix: 
[[103  13]
 [ 23  39]]
Accuracy: 
79.7752808988764
Report: 
              precision    recall  f1-score   support

           0       0.82      0.89      0.85       116
           1       0.75      0.63      0.68        62

    accuracy                           0.80       178
   macro avg       0.78      0.76      0.77       178
weighted avg       0.79      0.80      0.79       178



4it [00:00,  6.69it/s]

Confusion Matrix: 
[[100   9]
 [ 25  44]]
Accuracy: 
80.89887640449437
Report: 
              precision    recall  f1-score   support

           0       0.80      0.92      0.85       109
           1       0.83      0.64      0.72        69

    accuracy                           0.81       178
   macro avg       0.82      0.78      0.79       178
weighted avg       0.81      0.81      0.80       178

Confusion Matrix: 
[[90 15]
 [21 52]]
Accuracy: 
79.7752808988764
Report: 
              precision    recall  f1-score   support

           0       0.81      0.86      0.83       105
           1       0.78      0.71      0.74        73

    accuracy                           0.80       178
   macro avg       0.79      0.78      0.79       178
weighted avg       0.80      0.80      0.80       178



5it [00:00,  5.94it/s]

Confusion Matrix: 
[[92 11]
 [23 52]]
Accuracy: 
80.89887640449437
Report: 
              precision    recall  f1-score   support

           0       0.80      0.89      0.84       103
           1       0.83      0.69      0.75        75

    accuracy                           0.81       178
   macro avg       0.81      0.79      0.80       178
weighted avg       0.81      0.81      0.81       178



Confusion Matrix: 
[[101  15]
 [ 14  49]]
Accuracy: 
83.79888268156425
Report: 
              precision    recall  f1-score   support

           0       0.88      0.87      0.87       116
           1       0.77      0.78      0.77        63

    accuracy                           0.84       179
   macro avg       0.82      0.82      0.82       179
weighted avg       0.84      0.84      0.84       179

accuracies of Trees corresponding to 5 data splits :
tree-1 : 0.832
tree-2 : 0.798
tree-3 : 0.809
tree-4 : 0.798
tree-5 : 0.809
tree with best accuracy - 0.8324 :
{'Sex <= 0.5': [{'Pclass <= 2.5': [{'Age <= 2.5': [{'Parch <= 1.5': [1.0, 0.0]},
                                                   1.0]},
                                   {'Age <= 39.0': [{'SibSp <= 2.5': [{'Age <= 1.5': [1.0,
                                                                                      {'Age <= 3.0': [0.0,
                                                                                             

In [2]:
from sklearn.tree import DecisionTreeClassifier
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch']
sk_tree = DecisionTreeClassifier()
sk_tree.fit(train[features], train['label'])
print(sk_tree.predict(test[features]))
print(sk_tree.score(test[features], test['label'])*100)

[1 0 1 0 0 1 0 0 1 1 0 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1 0 1 0 1
 1 0 1 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 1 0 1 1 1 0 1
 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 1 0
 0 0 0 1 1 0 1 0 1 0 1 1 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 1 1 0 0 1 0 0
 0 0 1 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 1 0 0 1 0 1 1 1 0 1 0 1]
79.7752808988764
